In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
from plotly.plotly import iplot

import plotly
import boto3
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io


import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff


In [9]:
plotly.tools.set_credentials_file(username='', api_key='')

In [35]:
telcom = pd.read_csv("kaggle-telco-churn.csv") 
telcom.head()

#telcom.describe()
from pprint import pprint as pp

feature_uniquevals = [x for x in zip(telcom.columns.tolist(), telcom.nunique())]

print("Rows           : {}".format(telcom.shape[0]))
print("Columns        : {}".format(telcom.shape[1]))
print("Missing Values : {}".format(telcom.isnull().sum().values.sum()))
print("\nUnique\tFeature Name:")
for f, u in feature_uniquevals:
    print ("{}\t{}".format(u, f))

Rows           : 7043
Columns        : 21
Missing Values : 0

Unique	Feature Name:
7043	customerID
2	gender
2	SeniorCitizen
2	Partner
2	Dependents
73	tenure
2	PhoneService
3	MultipleLines
3	InternetService
3	OnlineSecurity
3	OnlineBackup
3	DeviceProtection
3	TechSupport
3	StreamingTV
3	StreamingMovies
3	Contract
2	PaperlessBilling
4	PaymentMethod
1585	MonthlyCharges
6531	TotalCharges
2	Churn


In [19]:
#Data Manipulation

#Replacing spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

#Dropping null values from total charges column which contain .15% missing data 
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

#convert to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'No internet service' : 'No'})
    
#replace values
telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

#Tenure to categorical column
def tenure_lab(telcom) :
    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                      axis = 1)

#Separating churn and non churn customers
churn     = telcom[telcom["Churn"] == "Yes"]
not_churn = telcom[telcom["Churn"] == "No"]

#Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

In [24]:
lab = telcom["Churn"].value_counts().keys().tolist()
val = telcom["Churn"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer attrition in data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)